In [4]:
import sys
sys.path.append('../')

In [9]:
from utils.utils import Utils
from utils.preprocess import Preprocess
import numpy as np
import pandas as pd

In [10]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sentiment_analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/eastwind/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [11]:
%%time
utils = Utils()
pre = Preprocess(mode="clean")

CPU times: user 2.63 s, sys: 71.4 ms, total: 2.71 s
Wall time: 2.71 s


In [13]:
%%time
data_emathy = utils.read_data("../dataset/train/train-empathy-distress-prediction-task-normalized.csv")
data_emathy

CPU times: user 24.8 ms, sys: 0 ns, total: 24.8 ms
Wall time: 28.7 ms


,message_id,response_id,article_id,empathy,distress,empathy_bin,distress_bin,essay,emotion,gender,...,personality_stability,iri_perspective_taking,iri_personal_distress,iri_fantasy,iri_empathatic_concern,gold_empathy,gold_distress,gold_emotion,gold_empathy_bin,gold_distress_bin
0,R_1hGrPtWM4SumG0U_1,R_1hGrPtWM4SumG0U,67,5.667,4.375,1,1,it is really disheartening to read about thes...,sadness,1,...,5.5,3.571,2.000,3.429,4.000,5.667,4.375,sadness,1,1
1,R_1hGrPtWM4SumG0U_2,R_1hGrPtWM4SumG0U,86,4.833,4.875,1,1,the phone lines from the suicide prevention l...,sadness,1,...,5.5,3.571,2.000,3.429,4.000,4.833,4.875,sadness,1,1
2,R_1hGrPtWM4SumG0U_3,R_1hGrPtWM4SumG0U,206,5.333,3.500,1,0,"no matter what your heritage , you should be ...",neutral,1,...,5.5,3.571,2.000,3.429,4.000,5.333,3.500,neutral,1,0
3,R_1hGrPtWM4SumG0U_4,R_1hGrPtWM4SumG0U,290,4.167,5.250,1,1,it is frightening to learn about all these sh...,fear,1,...,5.5,3.571,2.000,3.429,4.000,4.167,5.250,fear,1,1
4,R_1hGrPtWM4SumG0U_5,R_1hGrPtWM4SumG0U,342,5.333,4.625,1,1,the eldest generation of russians are not bei...,sadness,1,...,5.5,3.571,2.000,3.429,4.000,5.333,4.625,sadness,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1855,R_DHy2Rcz9Hym8jgl_1,R_DHy2Rcz9Hym8jgl,16,1.000,1.000,0,0,"These days , women are winning in sports . Th...",joy,2,...,7.0,5.000,1.286,4.286,4.429,1.000,1.000,joy,0,0
1856,R_DHy2Rcz9Hym8jgl_2,R_DHy2Rcz9Hym8jgl,158,1.000,1.000,0,0,I hate SIS . They are a group full of hate an...,anger,2,...,7.0,5.000,1.286,4.286,4.429,1.000,1.000,anger,0,0
1857,R_DHy2Rcz9Hym8jgl_3,R_DHy2Rcz9Hym8jgl,203,1.000,1.000,0,0,This is just disgusting . I cannot believe th...,disgust,2,...,7.0,5.000,1.286,4.286,4.429,1.000,1.000,disgust,0,0
1858,R_DHy2Rcz9Hym8jgl_4,R_DHy2Rcz9Hym8jgl,253,1.000,1.000,0,0,I feel like the world is so corrupt that it n...,sadness,2,...,7.0,5.000,1.286,4.286,4.429,1.000,1.000,sadness,0,0


In [84]:
%%time
data_gold = utils.read_data("../dataset/dev/dev-goldstandard.tsv")
data_gold

CPU times: user 6.4 ms, sys: 0 ns, total: 6.4 ms
Wall time: 6.49 ms


,empathy,distress,emotion
0,7.000,7.000,sadness
1,3.167,3.625,sadness
2,4.500,3.125,fear
3,2.000,2.000,anger
4,4.500,4.625,anger
...,...,...,...
265,2.000,2.250,anger
266,4.500,5.625,fear
267,1.833,2.375,neutral
268,1.833,1.375,joy


In [85]:
data_emathy["gold_empathy"] = data_gold["empathy"]
data_emathy["gold_distress"] = data_gold["distress"]
data_emathy["gold_emotion"] = data_gold["emotion"]
data_emathy

,message_id,response_id,article_id,essay,gender,education,race,age,income,personality_conscientiousness,...,personality_extraversion,personality_agreeableness,personality_stability,iri_perspective_taking,iri_personal_distress,iri_fantasy,iri_empathatic_concern,gold_empathy,gold_distress,gold_emotion
0,R_3QLVVnAgRBRH41U_1,R_3QLVVnAgRBRH41U,13,The story about the air strikes is very sadde...,1.0,4.0,3.0,20.0,24000.0,5.5,...,4.0,5.0,4.5,4.429,2.286,4.143,3.143,7.000,7.000,sadness
1,R_3QLVVnAgRBRH41U_2,R_3QLVVnAgRBRH41U,127,It is clear that climate change is something ...,1.0,4.0,3.0,20.0,24000.0,5.5,...,4.0,5.0,4.5,4.429,2.286,4.143,3.143,3.167,3.625,sadness
2,R_3QLVVnAgRBRH41U_3,R_3QLVVnAgRBRH41U,188,I did not know this comedian but thinking abo...,1.0,4.0,3.0,20.0,24000.0,5.5,...,4.0,5.0,4.5,4.429,2.286,4.143,3.143,4.500,3.125,fear
3,R_3QLVVnAgRBRH41U_4,R_3QLVVnAgRBRH41U,307,I am an affirmed believer that your punishmen...,1.0,4.0,3.0,20.0,24000.0,5.5,...,4.0,5.0,4.5,4.429,2.286,4.143,3.143,2.000,2.000,anger
4,R_3QLVVnAgRBRH41U_5,R_3QLVVnAgRBRH41U,409,"Okay , I hate hearing about disease outbreaks...",1.0,4.0,3.0,20.0,24000.0,5.5,...,4.0,5.0,4.5,4.429,2.286,4.143,3.143,4.500,4.625,anger
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,R_r3CoQ7YjIwxVvJT_1,R_r3CoQ7YjIwxVvJT,2,"While I feel bad for the arrest charges , I d...",2.0,2.0,1.0,37.0,93000.0,4.0,...,1.0,5.0,2.5,3.857,4.143,4.286,3.286,2.000,2.250,anger
266,R_r3CoQ7YjIwxVvJT_2,R_r3CoQ7YjIwxVvJT,131,This is such a tragedy . All I can think abou...,2.0,2.0,1.0,37.0,93000.0,4.0,...,1.0,5.0,2.5,3.857,4.143,4.286,3.286,4.500,5.625,fear
267,R_r3CoQ7YjIwxVvJT_3,R_r3CoQ7YjIwxVvJT,230,I am still on the fence regarding climate cha...,2.0,2.0,1.0,37.0,93000.0,4.0,...,1.0,5.0,2.5,3.857,4.143,4.286,3.286,1.833,2.375,neutral
268,R_r3CoQ7YjIwxVvJT_4,R_r3CoQ7YjIwxVvJT,326,I can understand why the Jewish community wou...,2.0,2.0,1.0,37.0,93000.0,4.0,...,1.0,5.0,2.5,3.857,4.143,4.286,3.286,1.833,1.375,joy


In [86]:
data_emathy["gold_empathy_bin"] = data_emathy["gold_empathy"].apply(lambda x: 1 if x >=4 else 0)
data_emathy["gold_distress_bin"] = data_emathy["gold_distress"].apply(lambda x: 1 if x >=4 else 0)
data_emathy

,message_id,response_id,article_id,essay,gender,education,race,age,income,personality_conscientiousness,...,personality_stability,iri_perspective_taking,iri_personal_distress,iri_fantasy,iri_empathatic_concern,gold_empathy,gold_distress,gold_emotion,gold_empathy_bin,gold_distress_bin
0,R_3QLVVnAgRBRH41U_1,R_3QLVVnAgRBRH41U,13,The story about the air strikes is very sadde...,1.0,4.0,3.0,20.0,24000.0,5.5,...,4.5,4.429,2.286,4.143,3.143,7.000,7.000,sadness,1,1
1,R_3QLVVnAgRBRH41U_2,R_3QLVVnAgRBRH41U,127,It is clear that climate change is something ...,1.0,4.0,3.0,20.0,24000.0,5.5,...,4.5,4.429,2.286,4.143,3.143,3.167,3.625,sadness,0,0
2,R_3QLVVnAgRBRH41U_3,R_3QLVVnAgRBRH41U,188,I did not know this comedian but thinking abo...,1.0,4.0,3.0,20.0,24000.0,5.5,...,4.5,4.429,2.286,4.143,3.143,4.500,3.125,fear,1,0
3,R_3QLVVnAgRBRH41U_4,R_3QLVVnAgRBRH41U,307,I am an affirmed believer that your punishmen...,1.0,4.0,3.0,20.0,24000.0,5.5,...,4.5,4.429,2.286,4.143,3.143,2.000,2.000,anger,0,0
4,R_3QLVVnAgRBRH41U_5,R_3QLVVnAgRBRH41U,409,"Okay , I hate hearing about disease outbreaks...",1.0,4.0,3.0,20.0,24000.0,5.5,...,4.5,4.429,2.286,4.143,3.143,4.500,4.625,anger,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,R_r3CoQ7YjIwxVvJT_1,R_r3CoQ7YjIwxVvJT,2,"While I feel bad for the arrest charges , I d...",2.0,2.0,1.0,37.0,93000.0,4.0,...,2.5,3.857,4.143,4.286,3.286,2.000,2.250,anger,0,0
266,R_r3CoQ7YjIwxVvJT_2,R_r3CoQ7YjIwxVvJT,131,This is such a tragedy . All I can think abou...,2.0,2.0,1.0,37.0,93000.0,4.0,...,2.5,3.857,4.143,4.286,3.286,4.500,5.625,fear,1,1
267,R_r3CoQ7YjIwxVvJT_3,R_r3CoQ7YjIwxVvJT,230,I am still on the fence regarding climate cha...,2.0,2.0,1.0,37.0,93000.0,4.0,...,2.5,3.857,4.143,4.286,3.286,1.833,2.375,neutral,0,0
268,R_r3CoQ7YjIwxVvJT_4,R_r3CoQ7YjIwxVvJT,326,I can understand why the Jewish community wou...,2.0,2.0,1.0,37.0,93000.0,4.0,...,2.5,3.857,4.143,4.286,3.286,1.833,1.375,joy,0,0


In [63]:
from scipy.stats import pearsonr

In [80]:
res = pearsonr(data_emathy["gold_distress_bin"].values.tolist(), data_emathy["distress_bin"].values.tolist())
res

(0.9999999999999998, 0.0)

In [87]:
data_emathy.to_csv("../dataset/dev/dev-empathy-distress-prediction-task-normalized.csv", index=False)

In [46]:
data.columns

Index(['message_id', 'response_id', 'empathy', 'distress', 'empathy_bin',
       'distress_bin', 'emotion_label', 'essay', 'gender', 'education', 'race',
       'age', 'income', 'personality_conscientiousness', 'personality_openess',
       'personality_extraversion', 'personality_agreeableness',
       'personality_stability', 'iri_perspective_taking',
       'iri_personal_distress', 'iri_fantasy', 'iri_empathatic_concern',
       'sentiment_score', 'vader_neg'],
      dtype='object')

In [ ]:
def generate_features(data):
    data.gender =  data.gender.apply(lambda x: 2 if x==5 else x)
    data["vader_scores"] = data.essay.apply(lambda x: [value for value in sentiment_analyzer.polarity_scores(x).values()])


In [114]:
import nltk
from nltk.corpus import stopwords

In [118]:
stop_words = stopwords.words('english')
stop_words

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [121]:
def get_frequencies(corpus, label, remove_stopwords=True):
    i=0
    freqs = {}       
    if remove_stopwords:
        for label, essay in zip(label, corpus):
            for word in essay.split():
                if word not in stop_words:
                    pair = (word, label)
                    if pair in freqs:
                        freqs[pair] +=1
                    else:
                        freqs[pair] = 1
    else:
        for label, essay in zip(label, corpus):
            for word in essay.split():
                pair = (word, label)
                if pair in freqs:
                    freqs[pair] +=1
                else:
                    freqs[pair] = 1
    return freqs

In [122]:
freqs = get_frequencies(data.essay.values.tolist(), 
                        data.empathy_bin.values.tolist(),
                        remove_stopwords=True)
freqs

{('really', 1): 274,
 ('disheartening', 1): 3,
 ('read', 1): 207,
 ('immigrants', 1): 8,
 ('article', 1): 289,
 ('drowned', 1): 3,
 ('.', 1): 4433,
 ('makes', 1): 94,
 ('feel', 1): 238,
 ('anxious', 1): 3,
 ('upset', 1): 13,
 ('whole', 1): 27,
 ('ordeal', 1): 2,
 ('happened', 1): 59,
 ('terrible', 1): 65,
 ('occurrence', 1): 3,
 ('happen', 1): 58,
 ('mediterranean', 1): 1,
 ('sea', 1): 15,
 ('thankfully', 1): 2,
 ('survivors', 1): 4,
 ('fact', 1): 39,
 ('babies', 1): 9,
 ('lost', 1): 57,
 ('much', 1): 119,
 ('emotional', 1): 13,
 ('phone', 1): 5,
 ('lines', 1): 3,
 ('suicide', 1): 17,
 ('prevention', 1): 2,
 ('line', 1): 10,
 ('surged', 1): 1,
 ('election', 1): 13,
 ('also', 1): 109,
 ('climbed', 1): 1,
 ('robin', 1): 1,
 ('williams', 1): 1,
 ('committed', 1): 7,
 ('help', 1): 141,
 ('need', 1): 156,
 ('suicidal', 1): 1,
 ('thoughts', 1): 13,
 ('condition', 1): 12,
 ('people', 1): 621,
 ('committing', 1): 6,
 ('permanent', 1): 3,
 ('solution', 1): 10,
 ('temporary', 1): 4,
 ('problem',

In [124]:
freqs.get(('problem', 0))

68

In [125]:
def get_bin_count(freqs):
    bin_1=0
    bin_0=0
    for key, value in freqs.items():
        if key[1] == 1:
            bin_1 += 1
        elif key[1] == 0:
            bin_0 += 1
    return bin_1, bin_0

In [126]:
bin_1, bin_0 = get_bin_count(freqs)
print(bin_1, bin_0)

7586 7133


In [25]:
def get_word_probabilities(freqs, bin_1, bin_0):
    word_probabilities = {}
    for key, value in freqs.items():
        pair = (key[0], key[1])
        p = 0
        if key[1] == 1:
            p = freqs.get(pair)/bin_1
        elif key[1] == 0:
            p = freqs.get(pair)/bin_0
        if p==0:
            word_probabilities[pair] = 0
        else:
            word_probabilities[pair] = np.log(p)
    return word_probabilities

In [26]:
# def get_subjectivity_scores(freqs):
#   epsilon = 1e-5
#   weights_dict = {}
#   for key in freqs.keys():
#     word = key[0]
#     if word not in weights_dict.keys():
#       neg = freqs.get((word, 'NEG'))
#       if neg==None:
#         neg = 0
#       pos = freqs.get((word, 'POS'))
#       if pos==None:
#         pos= 0
#       neu = freqs.get((word, 'NEU'))
#       if neu==None:
#         neu = 0
#       weights_neu = (neu - (pos + neg))/(neu + pos + neg + epsilon)
#       weights_posneg = (pos - neg)/(pos + neg + epsilon)
#       weights_dict[word] = (weights_neu, weights_posneg)
#   return weights_dict

In [ ]:
freqs = get_frequencies()